In [ ]:
%pylab inline
%matplotlib inline

import os
import sys
import numpy as np
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
current_path = !pwd
parent_path = os.path.dirname(current_path[0])
if parent_path not in sys.path: sys.path.append(parent_path)
work_path = "/".join(parent_path.split('/')[:-1])
if work_path not in sys.path: sys.path.append(work_path)
    
from DeepSparseCoding.data.dataset import Dataset
from DeepSparseCoding.analysis import analysis_picker as ap
from DeepSparseCoding.utils import data_processing as dp
import utils.model_handling as model_funcs
import utils.dataset_generation as iso_data
import utils.histogram_analysis as hist_funcs
import utils.plotting as pf

In [ ]:
text_width = 416.83269 #pt = 14.65cm
fontsize = 12
dpi = 800
font_settings = {
        "text.usetex": True,
        "font.family": "serif",
        "axes.labelsize": fontsize,
        "axes.titlesize": fontsize,
        "figure.titlesize": fontsize,
        "font.size": fontsize,
        "legend.fontsize": fontsize,
        "xtick.labelsize": fontsize-2,
        "ytick.labelsize": fontsize-2,
}
mpl.rcParams.update(font_settings)
mpl.pyplot.rc('text', usetex=True)

color_vals = dict(zip(["blk", "lt_green", "md_green", "dk_green", "lt_blue", "md_blue", "dk_blue", "lt_red", "md_red", "dk_red"],
  ["#000000", "#A9DFBF", "#196F3D", "#27AE60", "#AED6F1", "#3498DB", "#21618C", "#F5B7B1", "#E74C3C", "#943126"]))

In [ ]:
def get_dsc_activations_cell(analyzer, images, neuron, batch_size=10):
    '''
    Returns the activations from a model for given input images
    Parameters:
        analyzer [DSC analyzer object] an object from the DeepSparseCoding library
        images: an array with NumImages x W x H
        neuron: [int or vector of ints] that points to the neuron index
    Output:
        activations [np.ndarray] vector of length len(neuron)
    '''
    images = dp.reshape_data(images, flatten=analyzer.model.params.vectorize_data)[0]
    activations = analyzer.compute_activations(images, batch_size=None)[:, neuron]
    return activations

def load_analysis(params):
    params.model_dir = (work_path+"/Projects/"+params.model_name)
    analyzer = ap.get_analyzer(params.model_type)
    analyzer.setup(params)
    analyzer.model.setup(analyzer.model_params)
    analyzer.load_analysis(save_info=params.save_info)
    return params, analyzer

## Pre-trained LCA model

In [ ]:
class lca_1024_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_1024_vh"
    self.display_name = "Sparse Coding"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False
    self.use_group_activations = False

In [ ]:
params, analyzer = load_analysis(lca_1024_vh_params())
neuron_weights = [analyzer.bf_stats["basis_functions"][idx]
    for idx in range(len(analyzer.bf_stats["basis_functions"]))]

In [ ]:
cont_analysis = dict()

#cont_analysis["target_neuron_ids"] = get_rand_target_neuron_ids(3, len(meis["images"]))
cont_analysis["target_neuron_ids"] = [992, 381, 62]# [1, 2, 3]
cont_analysis["num_comparisons"] = 3
cont_analysis["min_angle"] = 5
cont_analysis["x_range"] = (-2.0, 2.0)
cont_analysis["y_range"] = (-2.0, 2.0)
cont_analysis["num_images"] = int(30**2)
cont_analysis["image_scale"] = 33 # norm of training data

iso_vectors = iso_data.compute_rand_vectors(
    [neuron_weights[idx] for idx in cont_analysis["target_neuron_ids"]],
    cont_analysis["num_comparisons"])
cont_analysis["rand_target_vectors"] = iso_vectors[0]
cont_analysis["rand_orth_vectors"] = iso_vectors[1]

iso_vectors = iso_data.compute_comp_vectors(
    neuron_weights,
    cont_analysis["target_neuron_ids"],
    cont_analysis["min_angle"],
    cont_analysis["num_comparisons"])
cont_analysis["comparison_neuron_ids"] = iso_vectors[0]
cont_analysis["comparison_target_vectors"] = iso_vectors[1]
cont_analysis["comparison_vectors"] = iso_vectors[2]

cont_analysis["target_vectors"] = cont_analysis["comparison_target_vectors"]

cont_analysis["contour_dataset"], datapoints = iso_data.get_contour_dataset(
    cont_analysis["target_vectors"],
    cont_analysis["comparison_vectors"],
    cont_analysis["x_range"],
    cont_analysis["y_range"],
    cont_analysis["num_images"],
    cont_analysis["image_scale"])

cont_analysis["activations"] = model_funcs.get_normalized_activations(
    analyzer,
    cont_analysis["target_neuron_ids"],
    datapoints,
    get_dsc_activations_cell)

In [ ]:
width_fraction = 1.0
show_contours = True
num_levels = 10
contour_fig, contour_handles = pf.plot_group_iso_contours(cont_analysis, num_levels, show_contours, text_width, width_fraction, dpi)

## Pre-trained Subspace-LCA model

In [ ]:
class lca_subspace_vh_params(object):
  def __init__(self):
    self.model_type = "lca_subspace"
    self.model_name = "lca_subspace_vh"
    self.display_name = "Subspace Sparse Coding"
    self.version = "5x_4_1.0_0.2"
    self.save_info = "analysis_train"
    self.overwrite_analysis_log = False
    self.use_group_activations = False

In [ ]:
params, analyzer = load_analysis(lca_subspace_vh_params())
neuron_weights = [analyzer.bf_stats["basis_functions"][idx]
    for idx in range(len(analyzer.bf_stats["basis_functions"]))]

In [ ]:
angle_matrix = iso_data.get_vector_angles(neuron_weights)[1]
min_angle = 10
num_below_min = np.count_nonzero(angle_matrix<min_angle) # many angles are -1 or 0
sorted_angle_indices = np.stack(np.unravel_index(np.argsort(angle_matrix.ravel()),
    angle_matrix.shape), axis=1)[num_below_min:, :]
#all_target_ids = sorted_angle_indices[:,0]
#target_neuron_ids = sorted_angle_indices[0:3, 0]


In [ ]:
num_target_neurons = 3
num_within_group_neurons = 1
num_outside_group_neurons = 2

unique_target_indices = list(set(sorted_angle_indices[:,0]))
selected_target_indices = [np.argwhere(sorted_angle_indices == unique_target_indices[idx])[0,0].item()
    for idx in range(num_target_neurons)]

In [ ]:
target_groups = analyzer.model.module.group_assignments[selected_target_indices[idx]
    for idx in range(num_target_neurons)]

within_group_ids = []
for group_index, group_number in enumerate(target_groups):
    sub_list = []
    for comp_neuron_idx in analyzer.model.module.group_ids[group_idx]:
        if comp_neuron_idx != selected_target_indices[group_index]:
            sub_list.append(comp_neuron_idx)
    within_group_ids.append(sub_list[:num_within_group_neurons])
    
outside_group_ids = []
for group_index, group_number in enumerate(target_groups):
    candidate_groups = [candidate_group_number
        for candidate_group_number in range(analyzer.model.module.num_groups)
        if candidate_group_number != group_number]
    candidate_comp_ids = []
    for candidate_group_number in candidate_groups:
        for neuron_idx in analyzer.model.module.group_ids[candidate_group_number]:
            candidate_comp_ids.append(neuron_idx)
    sub_list = []
    for candidate_group_number in candidate_groups:
        for comp_neuron_idx in analyzer.model.module.group_ids[candidate_group_number]:
            if angle_matrix[selected_target_indices[group_index], comp_neuron_idx] > cont_analysis["min_angle"]:
                if angle_matrix[selected_target_indices[group_index], comp_neuron_idx] < max_angle

In [ ]:
cont_analysis = dict()

cont_analysis["target_neuron_ids"] = [992, 381, 62]
cont_analysis["comparison_neuron_ids"] = [[993, 982, 950], [382, 991, 950], [64, 61, 200]]
cont_analysis["num_comparisons"] = 3
cont_analysis["min_angle"] = 5
cont_analysis["x_range"] = (-2.0, 2.0)
cont_analysis["y_range"] = (-2.0, 2.0)
cont_analysis["num_images"] = int(30**2)
cont_analysis["image_scale"] = 33 # norm of training data


iso_vectors = iso_data.compute_selected_vectors(
    neuron_weights,
    cont_analysis["target_neuron_ids"],
    cont_analysis["comparison_neuron_ids"])
cont_analysis["comparison_target_vectors"] = iso_vectors[1]
cont_analysis["comparison_vectors"] = iso_vectors[2]